In [1]:
import requests
from pathlib import Path
import sys
import pandas as pd
src_path = Path.cwd().parent / 'src'
sys.path.append(str(src_path))
import mle_utils as mUtil
import carball
from carball.json_parser.game import Game
from carball.analysis.analysis_manager import AnalysisManager, StatsManager

In [2]:
data_path = Path.cwd().parent / 'data'
secrets_path = data_path / 'secrets.txt'
secrets_file = open(str(secrets_path), mode='r')
auth_token = secrets_file.readline()
secrets_file.close()
s11_csv_path = data_path / 's11_raw_replay_data.csv'

pulling in s11 raw replay spreadsheet produced by mle stats team using ballchasing.com api

In [3]:
s11_spreadsheet = mUtil.read_multiindex_csv_to_df(s11_csv_path)

retreive all categories from the data

In [4]:
categories = []
for item in list(s11_spreadsheet.columns):
    if not item[0] in categories:
        categories.append(item[0])
categories

['Series',
 'Series Game',
 'Series Replay',
 'Team',
 'Team Stats Core',
 'Team Stats Ball',
 'Player Data',
 'Player ID',
 'Player Camera',
 'Player Stats Core',
 'Player Stats Boost',
 'Player Stats Movement',
 'Player Stats Positioning',
 'Player Stats Demo']

time to try and recreate the first 4 lines of the dataframe using the raw replay and the carball library

In [5]:
replay_filename = s11_spreadsheet['Series Replay'].iloc[0]['id'] + '.replay'
replay_filename

'6612902d-039b-4ec2-91e6-6036dcb7c0aa.replay'

In [6]:
cb_json = carball.decompile_replay(str(data_path / 's11_replays' / replay_filename))

initialize carball.game object with the replay file and create an analysis manager using that game

In [7]:
game = Game()
game.initialize(loaded_json=cb_json)

analysis_manager = AnalysisManager(game)
analysis_manager.create_analysis()
analysis_df = analysis_manager.get_data_frame()
analysis_df

Replay name not found
Player Sleegi as player has no MatchScore.
Score is not found for player
Ignoring player: Sleegi as player has no team.
Player SimpleAOB as player has no MatchScore.
Score is not found for player
Ignoring player: SimpleAOB as player has no team.
Player FOX | Thunderballs as player has no MatchScore.
Score is not found for player
Ignoring player: FOX | Thunderballs as player has no team.
Player RJsaurus as player has no MatchScore.
Score is not found for player
Ignoring player: RJsaurus as player has no team.
Dropping these columns[('game', 'is_overtime')]
The player never hit the ball during the "carry"
The player never hit the ball during the "carry"


FLA | TyTy                                                     \
            ping        pos_x        pos_y       pos_z        vel_x   
1              7   256.000000  3840.000000   17.049999     0.000000   
2              7   256.000000  3840.000000   17.049999     0.000000   
3              7   256.000000  3840.000000   17.049999     0.000000   
4              7   256.000000  3840.000000   17.049999     0.000000   
5              7   256.000000  3840.000000   17.049999     0.000000   
...          ...          ...          ...         ...          ...   
10608          7 -2537.729980  -759.869995  182.580002 -5120.399780   
10609          7 -2574.120117  -928.640015  192.750000 -4593.699951   
10610          7 -2574.120117  -928.640015  192.750000 -4593.699951   
10611          7 -2602.699951 -1079.449951  199.240005 -4019.599915   
10612          7 -2602.699951 -1079.449951  199.240005 -4019.599915   

                                                                       ...  \
              vel_y        vel_z  ang_vel_x    ang_vel_y    ang_vel_z  ...   
1          0.000000     3.200000   0.700000     0.000000     0.000000  ...   
2          0.000000     3.200000   0.700000     0.000000     0.000000  ...   
3          0.000000     3.200000   0.700000     0.000000     0.000000  ...   
4          0.000000     3.200000   0.700000     0.000000     0.000000  ...   
5          0.000000     3.200000   0.700000     0.000000     0.000000  ...   
...             ...          ...        ...          ...          ...  ...   
10608 -22367.700195  1569.799957 -88.999996   615.200005  3183.599854  ...   
10609 -22505.700684  1177.699966 -86.999998  1267.399979  3448.299866  ...   
10610 -22505.700684  1177.699966 -86.999998  1267.399979  3448.299866  ...   
10611 -22631.599121   806.500015 -30.699999  2286.100006  3267.699890  ...   
10612 -22631.599121   806.500015 -30.699999  2286.100006  3267.699890  ...   

             game                                                           \
             time     delta seconds_remaining replicated_seconds_remaining   
1      182.439346  0.037653               300                          NaN   
2      182.472687  0.033334               300                          NaN   
3      182.506027  0.033334               300                          NaN   
4      182.539368  0.033334               300                          NaN   
5      182.572708  0.033334               300                          NaN   
...           ...       ...               ...                          ...   
10608  596.774475  0.033335                 0                          NaN   
10609  596.810852  0.036405                 0                          NaN   
10610  596.844360  0.033561                 0                          NaN   
10611  596.877625  0.033334                 0                          NaN   
10612  596.915100  0.037517                 0                          NaN   

                                       FLA | TyTy  PAN // antzu PAN // Unborn  \
      ball_has_been_hit goal_number boost_collect boost_collect boost_collect   
1                  None         NaN           NaN           NaN           NaN   
2                  None         NaN           NaN           NaN           NaN   
3                  None         NaN           NaN           NaN           NaN   
4                  None         NaN           NaN           NaN           NaN   
5                  None         NaN           NaN           NaN           NaN   
...                 ...         ...           ...           ...           ...   
10608              True        -1.0           NaN           NaN           NaN   
10609              True        -1.0           NaN           NaN           NaN   
10610              True        -1.0           NaN           NaN           NaN   
10611              True        -1.0           NaN           NaN           NaN   
10612              True        -1.0           NaN           NaN           NaN   

using analysis_manager.get_json_data() seems to be the easiest way to retreive all of the stats in a easily manupilable format. Next I should identify where in the json data i need to extract data from and compile it into a single dictionary that can be read into a pandas dataframe for comparison to the original datasheet i was provided with.

In [9]:
proto_game = analysis_manager.get_protobuf_data()
#p_map = analysis_manager.stats_manager.

In [14]:
dict(analysis_manager.get_json_data()).keys()

dict_keys(['gameMetadata', 'players', 'teams', 'gameStats', 'parties', 'version', 'mutators'])

In [26]:
dict(analysis_manager.get_json_data())['teams'][0].keys()

dict_keys(['playerIds', 'score', 'isOrange', 'name', 'stats'])

creating a dictionary to map the appropriate json indeces to the s11_spreadsheet columns

In [38]:
tmp_cols = list(s11_spreadsheet.columns)
#running the line below will overwrite all the work done to create the atribute map
#atr_map = dict(zip(tmp_cols[21:], [None for i in range(0, len(tmp_cols[21:]))]))
atr_map

{('Series Replay', 'map_name'): None,
 ('Series Replay', 'id'): None,
 ('Series Replay', 'created'): None,
 ('Series Replay', 'match_guid'): None,
 ('Series Replay', 'title'): None,
 ('Series Replay', 'date'): None,
 ('Series Replay', 'duration'): None,
 ('Series Replay', 'overtime'): None,
 ('Series Replay', 'overtime_seconds'): None,
 ('Team', 'color'): None,
 ('Team', 'name'): None,
 ('Team Stats Core', 'goals'): None,
 ('Team Stats Core', 'goals_against'): None,
 ('Team Stats Ball', 'possession_time'): None,
 ('Team Stats Ball', 'time_in_side'): None,
 ('Player Data', 'name'): None,
 ('Player Data', 'start_time'): None,
 ('Player Data', 'end_time'): None,
 ('Player Data', 'car_name'): None,
 ('Player Data', 'car_id'): None,
 ('Player Data', 'steering_sensitivity'): None,
 ('Player ID', 'id'): None,
 ('Player ID', 'platform'): None,
 ('Player Camera', 'fov'): None,
 ('Player Camera', 'height'): None,
 ('Player Camera', 'pitch'): None,
 ('Player Camera', 'distance'): None,
 ('Player 

In [46]:
atr_map['Series Replay', 'map_name'] = ['gameMetadata', 'map']
atr_map

{('Series Replay', 'map_name'): ['gameMetadata', 'map'],
 ('Series Replay', 'id'): None,
 ('Series Replay', 'created'): None,
 ('Series Replay', 'match_guid'): None,
 ('Series Replay', 'title'): None,
 ('Series Replay', 'date'): None,
 ('Series Replay', 'duration'): None,
 ('Series Replay', 'overtime'): None,
 ('Series Replay', 'overtime_seconds'): None,
 ('Team', 'color'): None,
 ('Team', 'name'): None,
 ('Team Stats Core', 'goals'): None,
 ('Team Stats Core', 'goals_against'): None,
 ('Team Stats Ball', 'possession_time'): None,
 ('Team Stats Ball', 'time_in_side'): None,
 ('Player Data', 'name'): None,
 ('Player Data', 'start_time'): None,
 ('Player Data', 'end_time'): None,
 ('Player Data', 'car_name'): None,
 ('Player Data', 'car_id'): None,
 ('Player Data', 'steering_sensitivity'): None,
 ('Player ID', 'id'): None,
 ('Player ID', 'platform'): None,
 ('Player Camera', 'fov'): None,
 ('Player Camera', 'height'): None,
 ('Player Camera', 'pitch'): None,
 ('Player Camera', 'distance'

In [44]:
s11_spreadsheet['Series Replay']

,map_name,id,created,match_guid,title,date,duration,overtime,overtime_seconds
0,Salty Shores (Night),6612902d-039b-4ec2-91e6-6036dcb7c0aa,2020-08-31T05:12:03.630575Z,6D8E420C11EAEB2E58BE2595198661CE,199DE3CC491C08BDB02EF382E3EA03F6,2020-08-30T22:15:40Z,360,False,NaN
1,Salty Shores (Night),6612902d-039b-4ec2-91e6-6036dcb7c0aa,2020-08-31T05:12:03.630575Z,6D8E420C11EAEB2E58BE2595198661CE,199DE3CC491C08BDB02EF382E3EA03F6,2020-08-30T22:15:40Z,360,False,NaN
2,Salty Shores (Night),6612902d-039b-4ec2-91e6-6036dcb7c0aa,2020-08-31T05:12:03.630575Z,6D8E420C11EAEB2E58BE2595198661CE,199DE3CC491C08BDB02EF382E3EA03F6,2020-08-30T22:15:40Z,360,False,NaN
3,Salty Shores (Night),6612902d-039b-4ec2-91e6-6036dcb7c0aa,2020-08-31T05:12:03.630575Z,6D8E420C11EAEB2E58BE2595198661CE,199DE3CC491C08BDB02EF382E3EA03F6,2020-08-30T22:15:40Z,360,False,NaN
4,Salty Shores (Night),a7199b72-0ebd-4278-9809-46004ee5484d,2020-08-31T05:12:02.143048Z,0343DF7C11EAEB3004E52595C70BA343,3AB6E5684C5E255C43A489A93CD4569C,2020-08-30T22:24:03Z,355,False,NaN
...,...,...,...,...,...,...,...,...,...
16209,Mannfield (Night),c7a79f80-7bc6-4d44-ab66-074ddd5edaef,2020-11-17T08:12:04.669288Z,F04C82F811EB27A1FC4EA08AFA80569B,PAN4,2020-11-15T18:30:04Z,366,False,NaN
16210,Mannfield (Night),acaa46d0-534d-42b8-9a1f-f92346ec274d,2020-11-17T08:12:05.630368Z,E98F52E611EB27A2D7D9A08AE6D1B582,PAN5,2020-11-15T18:37:02Z,370,False,NaN
16211,Mannfield (Night),acaa46d0-534d-42b8-9a1f-f92346ec274d,2020-11-17T08:12:05.630368Z,E98F52E611EB27A2D7D9A08AE6D1B582,PAN5,2020-11-15T18:37:02Z,370,False,NaN
16212,Mannfield (Night),acaa46d0-534d-42b8-9a1f-f92346ec274d,2020-11-17T08:12:05.630368Z,E98F52E611EB27A2D7D9A08AE6D1B582,PAN5,2020-11-15T18:37:02Z,370,False,NaN


In [43]:
out_json = open(str(data_path / 'cb_wd' / 'first_replay.json'), 'x')
analysis_manager.write_json_out_to_file(out_json)
out_json.close()